In [ ]:
from block_ai.lib.myblokus import point
from block_ai.lib.myblokus.game_engine import GameEngine
from block_ai.lib.myblokus.game import Game, GameEnd
from block_ai.lib.myblokus.board import Board
from block_ai.lib.myblokus import point
from block_ai.lib.myblokus.piece import Piece
from block_ai.lib.myblokus.corner import Corner
from block_ai.lib.myblokus.orientation import Orientation
from block_ai.lib.myblokus.move import Move
from block_ai.lib.myblokus.agent import HumanInput, RandomAgent, Agent, PointAgent
from block_ai.lib.myblokus.board_view import BoardView


import itertools
import random
from copy import deepcopy
import numpy as np

from tqdm import tqdm_notebook

In [ ]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.WARNING)

%load_ext line_profiler

import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import matplotlib.patches as patches

In [ ]:
ge = GameEngine()
ge.play_game()

In [ ]:
for i in tqdm_notebook(range(1000)):
    ge = GameEngine()
    ge.play_game()

In [ ]:
def replay_game(moves):
    game = Game()
    try:
        for move in moves:
            game.make_move(move)
    except GameEnd:
        return game

In [ ]:
%lprun -f Board.are_squares_free replay_game(moves)

In [ ]:
moves = [Move(orientation=Orientation(((0, 0), (0, 1), (0, 2), (1, 0), (1, 1))),
                         player_id=0,
                         piece_id='p13',
                         corner=Corner(p1=(-1, -1), p2=(0, 0))),
 Move(orientation=Orientation(((0, 19), (1, 18), (1, 19), (2, 19), (3, 19))),
                         player_id=1,
                         piece_id='p15',
                         corner=Corner(p1=(-1, 20), p2=(0, 19))),
 Move(orientation=Orientation(((16, 18), (17, 18), (18, 18), (18, 19), (19, 19))),
                         player_id=2,
                         piece_id='p12',
                         corner=Corner(p1=(20, 20), p2=(19, 19))),
 Move(orientation=Orientation(((17, 0), (17, 1), (18, 0), (18, 1), (19, 0))),
                         player_id=3,
                         piece_id='p13',
                         corner=Corner(p1=(20, -1), p2=(19, 0))),
 Move(orientation=Orientation(((1, 4), (2, 2), (2, 3), (2, 4), (2, 5))),
                         player_id=0,
                         piece_id='p15',
                         corner=Corner(p1=(1, 1), p2=(2, 2))),
 Move(orientation=Orientation(((4, 18), (5, 17), (5, 18), (6, 17), (6, 18))),
                         player_id=1,
                         piece_id='p13',
                         corner=Corner(p1=(3, 19), p2=(4, 18))),
 Move(orientation=Orientation(((12, 17), (13, 17), (14, 17), (15, 17))),
                         player_id=2,
                         piece_id='p6',
                         corner=Corner(p1=(16, 18), p2=(15, 17))),
 Move(orientation=Orientation(((13, 2), (14, 2), (15, 2), (16, 2))),
                         player_id=3,
                         piece_id='p6',
                         corner=Corner(p1=(17, 1), p2=(16, 2))),
 Move(orientation=Orientation(((2, 8), (3, 6), (3, 7), (3, 8), (4, 7))),
                         player_id=0,
                         piece_id='p20',
                         corner=Corner(p1=(2, 5), p2=(3, 6))),
 Move(orientation=Orientation(((3, 16), (4, 15), (4, 16), (5, 15))),
                         player_id=1,
                         piece_id='p7',
                         corner=Corner(p1=(5, 17), p2=(4, 16))),
 Move(orientation=Orientation(((16, 16), (17, 15), (17, 16), (18, 15))),
                         player_id=2,
                         piece_id='p9',
                         corner=Corner(p1=(15, 17), p2=(16, 16))),
 Move(orientation=Orientation(((9, 1), (10, 1), (11, 1), (11, 2), (12, 1))),
                         player_id=3,
                         piece_id='p15',
                         corner=Corner(p1=(13, 2), p2=(12, 1))),
 Move(orientation=Orientation(((5, 8), (6, 8), (7, 8))),
                         player_id=0,
                         piece_id='p3',
                         corner=Corner(p1=(4, 7), p2=(5, 8))),
 Move(orientation=Orientation(((7, 15), (7, 16), (8, 16), (9, 16), (9, 17))),
                         player_id=1,
                         piece_id='p19',
                         corner=Corner(p1=(6, 17), p2=(7, 16))),
 Move(orientation=Orientation(((16, 12), (16, 13), (16, 14), (17, 12), (17, 13))),
                         player_id=2,
                         piece_id='p13',
                         corner=Corner(p1=(17, 15), p2=(16, 14))),
 Move(orientation=Orientation(((12, 3), (12, 4), (12, 5), (13, 5), (14, 5))),
                         player_id=3,
                         piece_id='p17',
                         corner=Corner(p1=(11, 2), p2=(12, 3))),
 Move(orientation=Orientation(((5, 6), (6, 6), (7, 6), (8, 5), (8, 6))),
                         player_id=0,
                         piece_id='p11',
                         corner=Corner(p1=(4, 7), p2=(5, 6))),
 Move(orientation=Orientation(((0, 16), (1, 15), (1, 16), (2, 15))),
                         player_id=1,
                         piece_id='p9',
                         corner=Corner(p1=(3, 16), p2=(2, 15))),
 Move(orientation=Orientation(((13, 11), (13, 12), (14, 10), (14, 11), (15, 11))),
                         player_id=2,
                         piece_id='p20',
                         corner=Corner(p1=(16, 12), p2=(15, 11))),
 Move(orientation=Orientation(((15, 6), (15, 7), (16, 7), (16, 8))),
                         player_id=3,
                         piece_id='p9',
                         corner=Corner(p1=(14, 5), p2=(15, 6))),
 Move(orientation=Orientation(((5, 2), (5, 3), (5, 4), (6, 4), (7, 4))),
                         player_id=0,
                         piece_id='p17',
                         corner=Corner(p1=(8, 5), p2=(7, 4))),
 Move(orientation=Orientation(((0, 14),)),
                         player_id=1,
                         piece_id='p1',
                         corner=Corner(p1=(1, 15), p2=(0, 14))),
 Move(orientation=Orientation(((13, 15), (14, 15), (15, 15))),
                         player_id=2,
                         piece_id='p3',
                         corner=Corner(p1=(16, 16), p2=(15, 15))),
 Move(orientation=Orientation(((17, 3), (18, 3), (18, 4), (19, 4))),
                         player_id=3,
                         piece_id='p7',
                         corner=Corner(p1=(16, 2), p2=(17, 3))),
 Move(orientation=Orientation(((1, 9), (1, 10), (1, 11), (2, 11), (2, 12))),
                         player_id=0,
                         piece_id='p12',
                         corner=Corner(p1=(2, 8), p2=(1, 9))),
 Move(orientation=Orientation(((10, 14), (10, 15), (11, 12), (11, 13), (11, 14))),
                         player_id=1,
                         piece_id='p12',
                         corner=Corner(p1=(9, 16), p2=(10, 15))),
 Move(orientation=Orientation(((15, 8), (15, 9), (16, 9), (16, 10))),
                         player_id=2,
                         piece_id='p7',
                         corner=Corner(p1=(14, 10), p2=(15, 9))),
 Move(orientation=Orientation(((10, 8), (10, 9), (11, 6), (11, 7), (11, 8))),
                         player_id=3,
                         piece_id='p12',
                         corner=Corner(p1=(12, 5), p2=(11, 6))),
 Move(orientation=Orientation(((8, 1), (8, 2), (8, 3), (9, 2), (10, 2))),
                         player_id=0,
                         piece_id='p16',
                         corner=Corner(p1=(7, 4), p2=(8, 3))),
 Move(orientation=Orientation(((3, 11), (3, 12), (3, 13), (3, 14))),
                         player_id=1,
                         piece_id='p6',
                         corner=Corner(p1=(4, 15), p2=(3, 14))),
 Move(orientation=Orientation(((10, 19), (11, 18), (11, 19), (12, 19))),
                         player_id=2,
                         piece_id='p8',
                         corner=Corner(p1=(12, 17), p2=(11, 18))),
 Move(orientation=Orientation(((17, 9), (17, 11), (18, 9), (18, 10), (18, 11))),
                         player_id=3,
                         piece_id='p14',
                         corner=Corner(p1=(16, 8), p2=(17, 9))),
 Move(orientation=Orientation(((6, 1),)),
                         player_id=0,
                         piece_id='p1',
                         corner=Corner(p1=(5, 2), p2=(6, 1))),
 Move(orientation=Orientation(((8, 10), (8, 11), (8, 12), (8, 13), (9, 13))),
                         player_id=1,
                         piece_id='p11',
                         corner=Corner(p1=(10, 14), p2=(9, 13))),
 Move(orientation=Orientation(((17, 6), (17, 7), (17, 8), (18, 6), (18, 8))),
                         player_id=2,
                         piece_id='p14',
                         corner=Corner(p1=(16, 9), p2=(17, 8))),
 Move(orientation=Orientation(((11, 10), (12, 9), (12, 10), (12, 11), (13, 9))),
                         player_id=3,
                         piece_id='p20',
                         corner=Corner(p1=(11, 8), p2=(12, 9))),
 Move(orientation=Orientation(((0, 13), (1, 13))),
                         player_id=0,
                         piece_id='p2',
                         corner=Corner(p1=(2, 12), p2=(1, 13))),
 Move(orientation=Orientation(((7, 19), (8, 18), (8, 19), (9, 19))),
                         player_id=1,
                         piece_id='p8',
                         corner=Corner(p1=(6, 18), p2=(7, 19))),
 Move(orientation=Orientation(((14, 3), (14, 4), (15, 4), (16, 4), (16, 5))),
                         player_id=2,
                         piece_id='p19',
                         corner=Corner(p1=(17, 6), p2=(16, 5))),
 Move(orientation=Orientation(((10, 3), (10, 4), (10, 5))),
                         player_id=3,
                         piece_id='p3',
                         corner=Corner(p1=(11, 6), p2=(10, 5))),
 Move(orientation=Orientation(((3, 1), (4, 0), (4, 1))),
                         player_id=0,
                         piece_id='p4',
                         corner=Corner(p1=(5, 2), p2=(4, 1))),
 Move(orientation=Orientation(((3, 9), (4, 8), (4, 9), (4, 10), (5, 10))),
                         player_id=1,
                         piece_id='p20',
                         corner=Corner(p1=(3, 11), p2=(4, 10))),
 Move(orientation=Orientation(((10, 16), (11, 15), (11, 16))),
                         player_id=2,
                         piece_id='p4',
                         corner=Corner(p1=(12, 17), p2=(11, 16))),
 Move(orientation=Orientation(((8, 7), (9, 6), (9, 7))),
                         player_id=3,
                         piece_id='p4',
                         corner=Corner(p1=(10, 8), p2=(9, 7))),
 Move(orientation=Orientation(((0, 7), (0, 8), (1, 6), (1, 7))),
                         player_id=0,
                         piece_id='p7',
                         corner=Corner(p1=(2, 8), p2=(1, 7))),
 Move(orientation=Orientation(((6, 12), (6, 13), (6, 14))),
                         player_id=1,
                         piece_id='p3',
                         corner=Corner(p1=(7, 15), p2=(6, 14))),
 Move(orientation=Orientation(((19, 14),)),
                         player_id=2,
                         piece_id='p1',
                         corner=Corner(p1=(18, 15), p2=(19, 14))),
 Move(orientation=Orientation(((17, 14), (18, 13), (18, 14), (19, 12), (19, 13))),
                         player_id=3,
                         piece_id='p18',
                         corner=Corner(p1=(18, 11), p2=(19, 12))),
 Move(orientation=Orientation(((8, 9), (9, 9), (9, 10), (10, 10), (10, 11))),
                         player_id=0,
                         piece_id='p18',
                         corner=Corner(p1=(7, 8), p2=(8, 9))),
 Move(orientation=Orientation(((12, 15), (12, 16))),
                         player_id=1,
                         piece_id='p2',
                         corner=Corner(p1=(11, 14), p2=(12, 15))),
 Move(orientation=Orientation(((19, 16), (19, 17))),
                         player_id=2,
                         piece_id='p2',
                         corner=Corner(p1=(18, 18), p2=(19, 17))),
 Move(orientation=Orientation(((19, 2),)),
                         player_id=3,
                         piece_id='p1',
                         corner=Corner(p1=(18, 3), p2=(19, 2))),
 Move(orientation=Orientation(((6, 10), (6, 11), (7, 10), (7, 11))),
                         player_id=0,
                         piece_id='p5',
                         corner=Corner(p1=(8, 9), p2=(7, 10))),
 Move(orientation=Orientation(((13, 14), (14, 14), (15, 12), (15, 13), (15, 14))),
                         player_id=1,
                         piece_id='p17',
                         corner=Corner(p1=(12, 15), p2=(13, 14))),
 Move(orientation=Orientation(((8, 14), (8, 15), (9, 14), (9, 15))),
                         player_id=2,
                         piece_id='p5',
                         corner=Corner(p1=(10, 16), p2=(9, 15))),
 Move(orientation=Orientation(((7, 0), (8, 0))),
                         player_id=3,
                         piece_id='p2',
                         corner=Corner(p1=(9, 1), p2=(8, 0))),
 Move(orientation=Orientation(((9, 0), (10, 0), (11, 0), (12, 0))),
                         player_id=0,
                         piece_id='p6',
                         corner=Corner(p1=(8, 1), p2=(9, 0))),
 Move(orientation=Orientation(((12, 7), (13, 6), (13, 7), (13, 8), (14, 7))),
                         player_id=2,
                         piece_id='p21',
                         corner=Corner(p1=(15, 8), p2=(14, 7))),
 Move(orientation=Orientation(((18, 7), (19, 6), (19, 7), (19, 8))),
                         player_id=3,
                         piece_id='p8',
                         corner=Corner(p1=(18, 9), p2=(19, 8))),
 Move(orientation=Orientation(((4, 11), (4, 12), (4, 13), (5, 12))),
                         player_id=0,
                         piece_id='p8',
                         corner=Corner(p1=(6, 11), p2=(5, 12))),
 Move(orientation=Orientation(((13, 1), (14, 0), (14, 1), (15, 0))),
                         player_id=0,
                         piece_id='p9',
                         corner=Corner(p1=(12, 0), p2=(13, 1)))]

In [ ]:
ge = GameEngine()
%lprun -f ge.game.add_corner_moves ge.play_game()

In [ ]:
game = Game()
pa = PointAgent(3)

## Without setters / getters

In [ ]:
%timeit Point(1, 1)

In [ ]:
%timeit (1, 1)

In [ ]:
289 / 11.9

## With setters / getters

In [ ]:
%timeit Point(1, 1)

In [ ]:
def get_adjacent(p):
    x, y = p
    return [
            (x - 1, y),
            (x, y - 1),
            (x, y + 1),
            (x + 1, y)
        ]

def get_corners(p):
    x, y = p
    return [
        (x - 1, y - 1),
        (x - 1, y + 1),
        (x + 1, y - 1),
        (x + 1, y + 1)
    ]

In [ ]:
import numpy as np

In [ ]:
p = np.array([3, 4])

m = np.matrix([[1, 0], [0, 1]])


In [ ]:
def rotate(m, p):
    return m * p

Game upgrades:
    moves contains duplicates
        dict of dicts vs dict with tuple of (corner, piece) key. latter's probably faster

    Is it really important to get all with a corner or all with a piece fast? maybe cut one out to speed up overall
  

Performance Tuning:

1. Write Game Test compare valid 

In [ ]:
def get_avg_moves_per_game():
    
    products = []

    for i in range(100):

        if i > 0 and i % 10 == 0:
            print(i)

        product = get_moves_per_game()
        products.append(product)
        
def get_moves_per_game():
    ge = GameEngine()
    move_counts = []
    while True:
        try:
            player_id = ge.game.player_pointer
            moves = list(ge.game.get_players_moves(player_id))
            move_counts.append((player_id, len(moves)))
            ge.play_turn()

        except GameEnd:
            break

    ids, options = zip(*move_counts)
    options = [float(o) for o in options]
    product = np.product(options)
    return product

In [ ]:
def show_orientation(o):
    """displays an orientation"""
    board = Board()
    vis = BoardView()
    offset = Point(10, 10)

    for i, j in itertools.product(range(20), range(20)):
        p = Point(i, j)
        val = -1
        if i < 10:
            if j < 10:
                val = 1
            else:
                val = 0
        else:
            if j < 10:
                val = 2
        board.assign(p, val)
            
    corners = o.get_corners()

    for c in corners:
        board.assign(c.p2 + offset, 4)
    new_points = [p + offset for p in o.points]
    for p in new_points:
        board.assign(p, 3)

    board.assign(Point(-1, -1), 6)
    board.display()

# ge = GameEngine(display=False)

%lprun -f ge.game.add_corner_moves ge.play_game()
ge.play_game()

In [ ]:
ge = GameEngine(display=False)

%lprun -f GameEngine.play_game ge.play_game()
ge.play_game()

50% of get corner moves involves rotating points. Are matracies faster?

Nagging Ideas:
     -  I should set the seeds for random players to or replay a game based on move history
        so that the runtimes are the same
    
     -  write tests so I don't break anything making optimizations

     -  Speed up BoardViewer 

Optimization Ideas:
     - lots of runtime spend rotating points. Are np.arrays / rot methods faster?
     
     - a bunch of time is spent converting orientations to sets in player.validate_move
         - raise try / catch is pretty expensive though it's nice to have reason why move is invalid
         - could use a boolean check + explain menthod (a little ugly)
  
     - logging can slow down stuff remove ones from the inner most loops

Fixed Logging

File: <ipython-input-74-8f1a337fe857>
Function: add_corner_moves at line 45

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
    45                                               def add_corner_moves(self, corner, player_id):
    46       336        358.0      1.1      0.0          player = self.players[player_id]
    47       336       3075.0      9.2      0.2          rotation = corner.get_rotation()
    48                                                   
    49       336        946.0      2.8      0.1          if not Board.on_board(corner.p2):
    50        45        159.0      3.5      0.0              logging.info("Corner %s is off board", corner)
    51        45         18.0      0.4      0.0              return
    52                                                   
    53       291       1214.0      4.2      0.1          logging.info("Adding corner %s moves", corner)
    54      4006       2874.0      0.7      0.2          for piece_id, piece in player.pieces.items():
    55                                                       
    56     20724      13778.0      0.7      0.8              for orientation in piece.orientations:
    57     17009      50509.0      3.0      3.0                  logging.debug("Examining orientation: %s", orientation)
    58                                                           
    59     17009     882254.0     51.9     52.7                  new_o = Orientation([corner.p2 + rotation(p) for p in orientation.points])
    60     17009      59013.0      3.5      3.5                  logging.debug("New orientation: %s", new_o)
    61     17009      33927.0      2.0      2.0                  m = Move(new_o, player_id, piece_id, corner)
    62                                           
    63     17009     567201.0     33.3     33.9                  if self.is_move_valid(m):
    64      6450      21146.0      3.3      1.3                      logging.debug("Adding Move: %s", m)
    65      6450      36560.0      5.7      2.2                      player.add_move(m)

Timer unit: 1e-06 s

Total time: 1.43679 s
File: <ipython-input-54-6bf5e323533c>
Function: add_corner_moves at line 45

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
    45                                               def add_corner_moves(self, corner, player_id):
    46       354        281.0      0.8      0.0          player = self.players[player_id]
    47       354       2668.0      7.5      0.2          rotation = corner.get_rotation()
    48                                                   
    49       354        796.0      2.2      0.1          if not Board.on_board(corner.p2):
    50        41        127.0      3.1      0.0              logging.info("Corner %s is off board", corner)
    51        41         19.0      0.5      0.0              return
    52                                                   
    53       313       1007.0      3.2      0.1          logging.info("Adding corner %s moves", corner)
    54      4174       2446.0      0.6      0.2          for piece_id, piece in player.pieces.items():
    55                                                       
    56     20946      10877.0      0.5      0.8              for orientation in piece.orientations:
    57     17085      43595.0      2.6      3.0                  logging.debug("Examining orientation: %s", orientation)
    58                                                           
    59     17085     716453.0     41.9     49.9                  new_o = Orientation([corner.p2 + rotation(p) for p in orientation.points])
    60     17085      46508.0      2.7      3.2                  logging.debug("New orientation: %s", new_o)
    61     17085      49656.0      2.9      3.5                  m = Move(new_o, player_id, piece_id, corner)
    62                                           
    63     17085     454759.0     26.6     31.7                  if self.is_move_valid(m):
    64      6220      78904.0     12.7      5.5                      logging.debug("Adding Move: %s", indent(m, 1))
    65      6220      28691.0      4.6      2.0                      player.add_move(m)

Timer unit: 1e-06 s

Total time: 1.43679 s
File: <ipython-input-54-6bf5e323533c>
Function: add_corner_moves at line 45

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
    45                                               def add_corner_moves(self, corner, player_id):
    46       354        281.0      0.8      0.0          player = self.players[player_id]
    47       354       2668.0      7.5      0.2          rotation = corner.get_rotation()
    48                                                   
    49       354        796.0      2.2      0.1          if not Board.on_board(corner.p2):
    50        41        127.0      3.1      0.0              logging.info("Corner %s is off board", corner)
    51        41         19.0      0.5      0.0              return
    52                                                   
    53       313       1007.0      3.2      0.1          logging.info("Adding corner %s moves", corner)
    54      4174       2446.0      0.6      0.2          for piece_id, piece in player.pieces.items():
    55                                                       
    56     20946      10877.0      0.5      0.8              for orientation in piece.orientations:
    57     17085      43595.0      2.6      3.0                  logging.debug("Examining orientation: %s", orientation)
    58                                                           
    59     17085     716453.0     41.9     49.9                  new_o = Orientation([corner.p2 + rotation(p) for p in orientation.points])
    60     17085      46508.0      2.7      3.2                  logging.debug("New orientation: %s", new_o)
    61     17085      49656.0      2.9      3.5                  m = Move(new_o, player_id, piece_id, corner)
    62                                           
    63     17085     454759.0     26.6     31.7                  if self.is_move_valid(m):
    64      6220      78904.0     12.7      5.5                      logging.debug("Adding Move: %s", indent(m, 1))
    65      6220      28691.0      4.6      2.0                      player.add_move(m)

```Third Run
Changed to set intersection instead itterative check almost the same time requirement```

Total time: 0.526981 s
File: <ipython-input-53-8fbb860b920e>
Function: validate_move at line 72

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
    72                                               def validate_move(self, move):
    73     72684      72960.0      1.0     13.8          if not self.has_piece(move.piece_id):
    74      1540      17668.0     11.5      3.4              raise RuntimeError(f"Move {move} invalid. Already played {move.piece_id}")
    75                                           
    76     71144     350042.0      4.9     66.4          new_points = {move.orientation.points}
    77     71144      49508.0      0.7      9.4          overlap = self.invalid_points.intersection(new_points)
    78     71144      36803.0      0.5      7.0          if len(overlap) > 0:
    79                                                       raise RuntimeError(f"Move {move} invalid. Includes invalid point {p}")

```Second Run
Used set instead list to hold invalid points```

Total time: 0.526422 s
File: <ipython-input-44-b012ab35adb8>
Function: validate_move at line 72

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
    72                                               def validate_move(self, move):
    73     48991      56822.0      1.2     10.8          if not self.has_piece(move.piece_id):
    74       833       9379.0     11.3      1.8              raise RuntimeError(f"Move {move} invalid. Already played {move.piece_id}")
    75                                           
    76    247885      96865.0      0.4     18.4          for p in move.orientation.points:
    77    204076     302546.0      1.5     57.5              if p in self.invalid_points:
    78      4349      60810.0     14.0     11.6                  raise RuntimeError(f"Move {move} invalid. Includes invalid point {p}")

```First Run```

Timer unit: 1e-06 s

Total time: 8.30674 s
File: <ipython-input-7-e9cda3e2ec9f>
Function: validate_move at line 72

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
    72                                               def validate_move(self, move):
    73     48443      74686.0      1.5      0.9          if not self.has_piece(move.piece_id):
    74       715      13962.0     19.5      0.2              raise RuntimeError(f"Move {move} invalid. Already played {move.piece_id}")
    75                                           
    76    239866     157205.0      0.7      1.9          for p in move.orientation.points:
    77    196660    7988919.0     40.6     96.2              if p in self.invalid_points:
    78      4522      71967.0     15.9      0.9                  raise RuntimeError(f"Move {move} invalid. Includes invalid point {p}")

## Status
Refactored but a bunch of cornsers get droped for no reason near the end

#### Questions:

why does it allow invalid moves in the below loop?

Theres an average branching factor of around 25
so call it 75 is that doable?





Number of chess game one guy says with 40 avg moves avg 30 moves per game
~ 10 ** 120 

James Hardy says (10 ** 50) ** 50 

Number of blockus game???

according to the game engine there are ~10^140 move options